# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [39]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [40]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位非常聪明的生物学家。你擅长从生物学的角度出发，回答各种生物学相关的问题，
并且能够将复杂的生物学概念解释得通俗易懂。而且还能够将生物学知识与其他学科联系起来，回答更加复杂的问题。
你能够分点回答每个问题。
问题如下:
{input}"""

computer_template = """你是一位非常聪明的计算机科学家。你擅长回答关于计算机科学的问题，
你也参加过一些业界尖端项目，有足够的项目履历和经验，你能够回答各种跟计算机相关的问题，并能够用通俗易懂的方式去回答和解释这些问题，
以及一些计算机概念。
问题如下:
{input}"""

chinese_template = """你是一位非常博学的汉语言文学老师。你擅长能够以直观，生动有趣，通俗易懂的方式去回答汉语言文学相关的问题，
并能够将抽象的概念解释的通俗易懂。你还能够把汉语言文学知识与其他学科联系起来，回答更加复杂的问题。
问题如下:
{input}"""

In [41]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
        {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
        {
        "name": "计算机",
        "description": "适用于回答计算机领域的问题",
        "prompt_template": computer_template,
    },
        {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学相关的问题",
        "prompt_template": chinese_template,
    },
]

In [42]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",base_url="https://api.xiaoai.plus/v1", api_key="sk-9o1als9Y9It7G1YoF38936F9061b4fE9B7C082Ec4cA83215")

In [43]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [44]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [45]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [46]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [47]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机领域的问题', '汉语言文学: 适用于回答汉语言文学相关的问题']


In [48]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机领域的问题
汉语言文学: 适用于回答汉语言文学相关的问题


In [49]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [50]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机领域的问题
汉语言文学: 适用于回答汉语言文学相关的问题

<< INPUT >>
{inp

In [51]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [52]:
print(chain.run("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.


黑体辐射是指处于热平衡状态下的理想物体，它能够在各种波长的电磁辐射中发出最大强度的辐射。这种辐射的强度和波长分布由温度决定，符合普朗克定律。黑体辐射在物理学中有重要的应用，例如研究热力学系统的能量分布和量子力学中的辐射问题。


In [53]:
print(
    chain.run(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.


首先，我们需要确定大于40的质数。根据质数的定义，大于40的质数必须是不能被2、3、5、7等小于它的质数整除的数。

接下来，我们可以列出所有大于40的质数：41、43、47、53、59、61、67、71、73、79、83、89、97。

然后，我们逐个检查这些质数加一后能否被3整除。我们可以发现，只有43加一后能被3整除，因此43是满足条件的第一个质数。

所以，答案是43。


In [54]:
router_chain.verbose = True

In [55]:
print(chain.run("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.


黑洞是一种极为密集的天体，具有非常强大的引力，它的引力场非常强大，连光都无法逃脱。它的存在是由于物质在极端的情况下被压缩成无限小的点，称为奇点。黑洞的边界被称为事件视界，它标志着光线无法逃脱的最远距离。黑洞的形成有多种情况，最常见的是恒星塌缩形成的中心天体。黑洞是宇宙中最神秘、最具挑战性的物体之一，它们也是研究宇宙和理解宇宙演化过程中的关键对象。


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [56]:
print(chain.run("普通细胞和癌细胞有哪些区别?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '普通细胞和癌细胞有哪些区别?'}
> Finished chain.

普通细胞和癌细胞是两种不同的细胞类型。普通细胞是身体各个组织和器官的基本组成单位，具有生长、分裂和死亡的能力，可以完成各种生理功能。而癌细胞则是一种异常细胞，它们失去了正常细胞的生长和分裂控制机制，导致无限制的增殖和扩散，最终形成肿瘤。以下是普通细胞和癌细胞的主要区别：

1. 细胞形态：普通细胞通常具有规则的形态，大小和形状相似，细胞核也较为规则。而癌细胞则具有较大的形态多样性，大小和形状不规则，细胞核也可能异常增


In [57]:
print(chain.run("计算机架构中ARM和x86有什么区别？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '计算机架构中ARM和x86有什么区别？'}
> Finished chain.


ARM和x86是两种不同的计算机架构，它们有如下几点区别：

1. 指令集：指令集是计算机处理器能够识别和执行的命令集合。ARM和x86都有自己独特的指令集。ARM指令集主要用于移动设备和嵌入式系统，而x86指令集则主要用于个人电脑和服务器。

2. 处理器设计：ARM和x86的处理器架构也有所不同。ARM处理器一般采用精简指令集(RISC)架构，它通过减少指令集的复杂性来提高处理器的性能。而x86处理器则采用复杂指令集(CISC)架构，它可以执行更复杂的指令，但相应的处理器设计也更复杂。

3. 功耗：由于ARM处理


In [58]:
print(chain.run("在三国演义中，诸葛亮为何挥泪斩马谡？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '在三国演义中，诸葛亮为何挥泪斩马谡？'}
> Finished chain.
根据小说《三国演义》的描述，诸葛亮挥泪斩马谡是因为马谡在汉中之战中表现出了不负责任的行为。马谡被任命为汉中太守，但却没有采取有效的措施防御蜀国的敌人。这导致了汉中失陷，蜀国的重要战略地位受到了威胁。诸葛亮为了维护蜀国的利益，不得不斩断马谡的头颅以示惩罚。
